In [6]:
import pandas as pd
import pathlib
import sys
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
from keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from keras.layers import GRU, Dense, Embedding, Flatten, Dropout
from keras.activations import softmax

In [7]:
# Load tokenizer configuration and vocabulary from JSON file
with open('D:\project\AWS\Emotional_analysis_NLP\data\interim\\tokenizer.json', 'r') as f:
    tokenizer_json = json.load(f)
tokenizer = tokenizer_from_json(tokenizer_json)

In [8]:
# data split into feature and target
train = pd.read_csv('D:\project\AWS\Emotional_analysis_NLP\data\processed\\train.csv')
train.head()

,text,label
0,ive blabbed enough tonight im tired ive feelin...,0
1,woke really early morning drove feel ecstatic ...,1
2,feel never gave rest day megabrick feeling stu...,3
3,feeling restless teary flat sad strange today,4
4,feel like im doomed ive even began,0


In [9]:
X_train = train['text'].astype(str)
y_train = train['label']

In [10]:
test = pd.read_csv('D:\project\AWS\Emotional_analysis_NLP\data\processed\\test.csv')
test.head()

,text,label
0,id say maybe made feel foolish would reeeeeeal...,0
1,joined lds church admit feeling somewhat asham...,0
2,must admit didnt feel like hugging angry disgu...,3
3,hate still feel nerve damaged badly enough oft...,0
4,im actually feeling little smug,1


In [11]:
X_test = test['text'].astype(str)
y_test = test['label']

In [12]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (333447,)
X_test shape: (83362,)
y_train shape: (333447,)
y_test shape: (83362,)


In [13]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [14]:
# Max Len in X_train_sequences
maxlen = max(len(tokens) for tokens in X_train_sequences)
print("Maximum sequence length (maxlen):", maxlen)

Maximum sequence length (maxlen): 79


In [15]:
# Perform padding on X_train and X_test sequences
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=maxlen, padding='post')

In [16]:
X_test_padded 

array([[  123,    33,   172, ...,     0,     0,     0],
       [ 2941, 12893,   748, ...,     0,     0,     0],
       [  194,   310,    48, ...,     0,     0,     0],
       ...,
       [  851,   276,  1128, ...,     0,     0,     0],
       [ 1874,  6225,   645, ...,     0,     0,     0],
       [   83,     1,   401, ...,     0,     0,     0]])

In [17]:
# Print the padded sequences for X_train and X_test
print("X_train_padded:")
print(X_train_padded)
print("\nX_test_padded:")
print(X_test_padded)

X_train_padded:
[[   20 26719    73 ...     0     0     0]
 [  275     5   738 ...     0     0     0]
 [    1    45   735 ...     0     0     0]
 ...
 [    1   606     0 ...     0     0     0]
 [    4    18   476 ...     0     0     0]
 [    1   149  1686 ...     0     0     0]]

X_test_padded:
[[  123    33   172 ...     0     0     0]
 [ 2941 12893   748 ...     0     0     0]
 [  194   310    48 ...     0     0     0]
 ...
 [  851   276  1128 ...     0     0     0]
 [ 1874  6225   645 ...     0     0     0]
 [   83     1   401 ...     0     0     0]]


In [18]:
print(f'X_train shape: {X_train_padded.shape}')
print(f'X_test shape: {X_test_padded.shape}')

X_train shape: (333447, 79)
X_test shape: (83362, 79)


# Vocabulary size

In [19]:
# Embedding Vocabulary Size 
vocabulary_size = len(set(token for sequence in X_train_padded for token in sequence))
vocabulary_size

49510

In [20]:
vocabulary_size = len(tokenizer.word_index) + 1  # Add 1 for the padding token


# Model building
- Sequential: This initializes a sequential model, allowing you to build the model layer by layer in a linear fashion.¶
- Embedding layer: This layer is responsible for converting integer-encoded words into dense vectors of fixed size. It takes three parameters: input_dim, which specifies the size of the vocabulary (in this case, vocabulary_size), output_dim, which determines the dimension of the dense embedding, and input_length, which specifies the length of input sequences (in this case, maxlen).
- Bidirectional GRU layer: This layer adds a bidirectional Gated Recurrent Unit (GRU) layer with 128 units. GRU is a type of recurrent neural network (RNN) that is similar to LSTM but has fewer parameters. The bidirectional wrapper allows the GRU to process input sequences in both forward and backward directions, enhancing the model's ability to capture long-range dependencies in the data.
- Batch normalization layer: Batch normalization is a technique used to improve the training speed and stability of neural networks by normalizing the activations of each layer. It helps mitigate the internal covariate shift problem.
- Dropout regularization: Dropout is a regularization technique that randomly sets a fraction of input units to zero during training, which helps prevent overfitting by reducing the reliance on specific neurons.
- Dense layer with ReLU activation: This layer adds a fully connected dense layer with 64 units and Rectified Linear Unit (ReLU) activation function. ReLU is a non-linear activation function that introduces non-linearity to the model, allowing it to learn complex patterns in the data.
- Dropout regularization: Another dropout layer is added after the dense layer to further prevent overfitting.
- Output layer with softmax activation: The final layer is a dense layer with 6 units (assuming there are 6 classes) and softmax activation function. Softmax activation normalizes the output vector into a probability distribution over the classes, making it suitable for multi-class classification problems.¶

In [21]:
# Define the model
model = Sequential()

# Add an embedding layer with input_dim=1000, output_dim=100, input_length=75
model.add(Embedding(input_dim=vocabulary_size, output_dim=100,input_shape=(79,)))

# Add a bidirectional GRU layer with 128 units
model.add(Bidirectional(GRU(128)))

# Add batch normalization layer
model.add(BatchNormalization())

# Add dropout regularization
model.add(Dropout(0.5))

# Add a dense layer with 64 units and ReLU activation
model.add(Dense(64, activation='relu'))

# Add dropout regularization
model.add(Dropout(0.5))

# Add the output layer with 6 units for 6 labels and softmax activation
model.add(Dense(6, activation='softmax'))
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Print model summary
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 75, 100)           6763500   
                                                                 
 bidirectional (Bidirection  (None, 256)               176640    
 al)                                                             
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dropout_1 (Dropout)         (None, 64)               

In [22]:
# Model Train 
history = model.fit(X_train_padded, y_train,
                     epochs=1, batch_size=1500,
                       validation_data=(X_test_padded, y_test))

ValueError: in user code:

    File "c:\Users\kunal\anaconda3\envs\Emo\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\kunal\anaconda3\envs\Emo\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\kunal\anaconda3\envs\Emo\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\kunal\anaconda3\envs\Emo\Lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\kunal\anaconda3\envs\Emo\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\kunal\anaconda3\envs\Emo\Lib\site-packages\keras\src\engine\input_spec.py", line 298, in 

# load model

In [7]:
import json
from tensorflow.keras.models import model_from_json

In [4]:
from tensorflow.keras.models import load_model
loaded_model = load_model('D:/project/AWS/Emotional_analysis_NLP/models/model.keras')# Load the pre-trained sentiment analysis model


TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 75], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_1'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [97]:
y_pred = model.predict(X_test_padded)

2606/2606 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step


In [98]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [89]:
y_pred 

array([0, 0, 2, ..., 0, 2, 0], dtype=int64)

In [103]:
loaded_model 

NameError: name 'loaded_model' is not defined

In [3]:
keras.__version__

'2.15.0'

In [1]:
import keras
import tensorflow

In [2]:
tensorflow.__version__

'2.15.1'